## Custom models

To score your model, simply wrap it in one of the available wrappers (PyTorch, TensorFlow or Keras) and pass it to a scoring method.

### PyTorch

Let's define a simple convolution-relu-linear-relu model.

In [1]:
import numpy as np
import torch
from torch import nn

from candidate_models.models.implementations.pytorch import PytorchModel


class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = torch.nn.Conv2d(in_channels=3, out_channels=2, kernel_size=3)
        self.relu1 = torch.nn.ReLU()
        linear_input_size = np.power((224 - 3 + 2 * 0) / 1 + 1, 2) * 2
        self.linear = torch.nn.Linear(int(linear_input_size), 1000)
        self.relu2 = torch.nn.ReLU()  # can't get named ReLU output otherwise

        # init weights for reproducibility
        self.conv1.weight.data.fill_(0.01)
        self.conv1.bias.data.fill_(0.01)
        self.linear.weight.data.fill_(0.01)
        self.linear.bias.data.fill_(0.01)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = x.view(x.size(0), -1)
        x = self.linear(x)
        x = self.relu2(x)
        return x


Using TensorFlow backend.


Next, wrap the model in the PyTorch wrapper.

In [2]:
from candidate_models.models.implementations.pytorch import PytorchModel


class MyModelWrapper(PytorchModel):
    def _create_model(self, weights):
        my_model = MyModel()
        assert weights is None  # weight loading would go here
        return my_model

Finally, pass the model to the scoring method and pass an identifier (for caching results).

In [3]:
from candidate_models import score_model

score = score_model(model_identifier='test_pytorch', model=MyModelWrapper, 
                    layers=['linear', 'relu2'], weights=None, pca_components=None)
print(score)

Score(_variable=<xarray.Variable (aggregation: 1)>
array([0.247097])
Attributes:
    raw:      Score(_variable=<xarray.Variable (benchmark: 2, layer: 2, aggre...,_coords=OrderedDict([('aggregation', <xarray.IndexVariable 'aggregation' (aggregation: 1)>
array(['center'], dtype='<U6'))]),_name=None,_file_obj=None,_initialized=True)


The above returns an aggregate Brain-Score of multiple benchmarks.
For scores on individual benchmarks, see the "Score" section at the end.

### TensorFlow Slim

Let's define a simple model with a convolution and pooling.

In [4]:
import tensorflow as tf
slim = tf.contrib.slim


def _create_tf_model(inputs):
    with tf.variable_scope('my_model', values=[inputs]) as sc:
        end_points_collection = sc.original_name_scope + '_end_points'
        # Collect outputs for conv2d, fully_connected and max_pool2d.
        with slim.arg_scope([slim.conv2d, slim.fully_connected, slim.max_pool2d],
                            outputs_collections=[end_points_collection]):
            net = slim.conv2d(inputs, 64, [11, 11], 4, padding='VALID', scope='conv1')
            net = slim.max_pool2d(net, [5, 5], 5, scope='pool1')
            net = slim.max_pool2d(net, [3, 3], 2, scope='pool2')
            end_points = slim.utils.convert_collection_to_dict(end_points_collection)
            return net, end_points

Next, wrap the model in the TensorFlow-slim wrapper.

In [5]:
from candidate_models.models.implementations.tensorflow_slim import TensorflowSlimModel
# requires you to have TF slim installed: https://github.com/tensorflow/tensorflow/tree/master/tensorflow/contrib/slim
from preprocessing import vgg_preprocessing


class MyModelWrapper(TensorflowSlimModel):
    def _create_inputs(self, batch_size, image_size):
        inputs = tf.placeholder(dtype=tf.float32, shape=[batch_size, image_size, image_size, 3])
        preprocess_image = vgg_preprocessing.preprocess_image
        return tf.map_fn(lambda image: preprocess_image(tf.image.convert_image_dtype(image, dtype=tf.uint8),
                                                        image_size, image_size), inputs)

    def _create_model(self, inputs):
        return _create_tf_model(inputs)

    def _restore(self, weights):
        assert weights is None
        init = tf.initialize_all_variables()
        self._sess.run(init)


Finally, pass the model to the scoring method and pass an identifier (for caching results).

In [6]:
from candidate_models import score_model

score = score_model(model_identifier='test_tensorflow_slim', model=MyModelWrapper, 
                    layers=['my_model/pool2'], weights=None, pca_components=None)
print(score)

Score(_variable=<xarray.Variable (aggregation: 1)>
array([0.424662])
Attributes:
    raw:      Score(_variable=<xarray.Variable (benchmark: 2, layer: 1, aggre...,_coords=OrderedDict([('aggregation', <xarray.IndexVariable 'aggregation' (aggregation: 1)>
array(['center'], dtype='<U6'))]),_name=None,_file_obj=None,_initialized=True)


The above returns an aggregate Brain-Score of multiple benchmarks with individual benchmark scores in the raw values.

## Pre-defined models

### Single model

Scoring a model on neural data can be done in a single line using the `score_physiology` method.
This call is agnostic of a specific model implementation, it will simply look up the model name 
in `neurality.models.models` and use the implementation defined there (also see `examples/model-activations.ipynb`).
By default, the pre-defined layers of a model will be used to retrieve the activations, 
but you can also pass your own.
Just like with the model implementations, the result of this method call will be cached 
so that it only needs to be computed once.


In [7]:
from candidate_models import score_model

score = score_model(model='alexnet')
print(score)


Score(_variable=<xarray.Variable (aggregation: 1)>
array([0.60981])
Attributes:
    raw:      Score(_variable=<xarray.Variable (benchmark: 2, layer: 7, aggre...,_coords=OrderedDict([('aggregation', <xarray.IndexVariable 'aggregation' (aggregation: 1)>
array(['center'], dtype='<U6'))]),_name=None,_file_obj=None,_initialized=True)


## Score aggregate and raw values

A score typically comes with an estimate of the center (e.g. mean) and error (e.g. standard error of the mean).
These values are aggregations over splits and often neuroids.

We can also retrieve the raw scores from the same object, using `.attrs['raw']`.
For the Brain-Score, these contain scores for individual layers on individual benchmarks.

In [8]:
from candidate_models import score_model

score = score_model(model='alexnet')
benchmark_values = score.attrs['raw']
print(benchmark_values)

Score(_variable=<xarray.Variable (benchmark: 2, layer: 7, aggregation: 2)>
array([[[0.376876, 0.006124],
        [0.512881, 0.003886],
        [0.537398, 0.003578],
        [0.548949, 0.003162],
        [0.588876, 0.00288 ],
        [0.559185, 0.003265],
        [0.51445 , 0.002867]],

       [[0.625188, 0.004513],
        [0.630744, 0.004208],
        [0.626941, 0.003936],
        [0.574096, 0.00437 ],
        [0.48632 , 0.006042],
        [0.425624, 0.005558],
        [0.347992, 0.008119]]])
Attributes:
    raw:      <xarray.DataAssembly (benchmark: 2, layer: 7, split: 10, neuroi...,_coords=OrderedDict([('benchmark', <xarray.IndexVariable 'benchmark' (benchmark: 2)>
array(['dicarlo.Majaj2015.IT', 'dicarlo.Majaj2015.V4'], dtype=object)), ('layer', <xarray.IndexVariable 'layer' (layer: 7)>
array(['features.2', 'features.5', 'features.7', 'features.9', 'features.12',
       'classifier.2', 'classifier.5'], dtype=object)), ('aggregation', <xarray.IndexVariable 'aggregation' (aggregation:

The raw values are just another xarray object and as such, we can easily filter.

In [9]:
print(benchmark_values.sel(benchmark='dicarlo.Majaj2015.V4', layer='features.2'))

Score(_variable=<xarray.Variable (aggregation: 2)>
array([0.625188, 0.004513])
Attributes:
    raw:      <xarray.DataAssembly (split: 10, neuroid: 256)>\narray([[     n...,_coords=OrderedDict([('benchmark', <xarray.Variable ()>
array('dicarlo.Majaj2015.V4', dtype='<U20')), ('layer', <xarray.Variable ()>
array('features.2', dtype='<U10')), ('aggregation', <xarray.IndexVariable 'aggregation' (aggregation: 2)>
array(['center', 'error'], dtype='<U6'))]),_name=None,_file_obj=None,_initialized=True)


For individual benchmarks, the raw values contain single correlations over neuroids and splits.
Since Brain-Score's raw values are the values for individual benchmarks, they contain another raw values attribute which you can use to retrieve the per-split, per-neuroid values.

In [10]:
raw_scores = benchmark_values.attrs['raw']
print(raw_scores)

<xarray.DataAssembly (benchmark: 2, layer: 7, split: 10, neuroid: 256)>
array([[[[0.368393, ...,      nan],
         ...,
         [0.361794, ...,      nan]],

        ...,

        [[0.344986, ...,      nan],
         ...,
         [0.310163, ...,      nan]]],


       [[[     nan, ..., 0.375042],
         ...,
         [     nan, ..., 0.401341]],

        ...,

        [[     nan, ..., 0.515528],
         ...,
         [     nan, ..., 0.475515]]]])
Coordinates:
  * benchmark   (benchmark) object 'dicarlo.Majaj2015.IT' 'dicarlo.Majaj2015.V4'
  * neuroid     (neuroid) MultiIndex
  - neuroid_id  (neuroid) object 'Chabo_L_A_2_4' 'Chabo_L_A_3_3' ...
  - arr         (neuroid) object 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' ...
  - col         (neuroid) int64 4 3 5 0 1 2 3 4 5 6 2 3 5 0 1 2 3 4 5 6 1 2 ...
  - hemisphere  (neuroid) object 'L' 'L' 'L' 'L' 'L' 'L' 'L' 'L' 'L' 'L' 'L' ...
  - subregion   (neuroid) object 'cIT' 'cIT' 'aIT' 'cIT' 'cIT' 'cIT' 'cIT' ...
  - animal      (neuroid